# An example of neutrino propagation simulation through flexOPT.jl 
Some of functions are the fruit of works from Estelle Salomé (July-August 2025) and TIPE (2025-2026).

i need to make a lazy option

In [1]:

using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using Colors



  Activating 

  2.016087 seconds (3.83 M allocations: 173.433 MiB, 14.95% gc time, 99.98% compilation time)


project at `~/Documents/Github/flexOPT`


data directory which is local:

In [2]:
dir="/Users/nobuaki/Documents/MantleConvectionTakashi/op_old_full_mars_2025/"

"/Users/nobuaki/Documents/MantleConvectionTakashi/op_old_full_mars_2025/"

## making 2D(/3D) box with the aid of GeoPoint.jl 

Please refer to getRegionalBox.ipynb if you have some questions 

In [4]:
set_default_planet!(:SphericalEarth) # SphericalEarth means a perfect sphere with a radius of 6371000.0 m 
# otherwise of course, I made an Earth with ellipticity

Ellipsoid(SphericalEarth)

In [5]:
# you can do some more exercises with getRegionalBox.ipynb but just trust me here

# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(0.0,0.0;alt=30.e3) # lat, lon in degrees, altitude in metre (as an option)
# if you call GeoPoints with three floats without ; for alt, it will interpret as x, y, z in cartesian ecef coordinates
p2 = GeoPoint(0.0,180.0;alt=30.e3) # 


Δx = 25.e3 # in metre
Δz = 25.e3

altMax = 6400.e3
altMin = -6400.e3

-6.4e6

In [6]:
boxGrids=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax;centreOption ="centreOfPlanet")

centreOption = "centreOfPlanet"


(allGridsInGeoPoints = GeoPoint[GeoPoint(-90.0, 0.0, 29000.0, [0.0, 0.0, -6.4e6], 6.4e6) GeoPoint(-90.0, 0.0, 4000.0, [0.0, 0.0, -6.375e6], 6.375e6) … GeoPoint(90.0, 0.0, 4000.0, [0.0, 0.0, 6.375e6], 6.375e6) GeoPoint(90.0, 0.0, 29000.0, [0.0, 0.0, 6.4e6], 6.4e6); GeoPoint(-89.77618949963147, 180.0, 29048.827938737348, [-25000.0, 0.0, -6.4e6], 6.400048827938737e6) GeoPoint(-89.77531182037549, 180.0, 4049.0194193804637, [-25000.0, 0.0, -6.375e6], 6.37504901941938e6) … GeoPoint(89.77531182037549, 180.0, 4049.0194193804637, [-25000.0, 0.0, 6.375e6], 6.37504901941938e6) GeoPoint(89.77618949963147, 180.0, 29048.827938737348, [-25000.0, 0.0, 6.4e6], 6.400048827938737e6); … ; GeoPoint(-26.609883546263998, 180.0, 7.917478750377873e6, [-1.2775e7, 0.0, -6.4e6], 1.4288478750377873e7) GeoPoint(-26.52018370033453, 180.0, 7.906298413915709e6, [-1.2775e7, 0.0, -6.375e6], 1.4277298413915709e7) … GeoPoint(26.52018370033453, 180.0, 7.906298413915709e6, [-1.2775e7, 0.0, 6.375e6], 1.4277298413915709e7) Ge

In [ ]:
# getting model parameters on the 2D disk
seismicModel=lazyProduceOrLoad("seismicModel2DGlobal",getParamsWithoutTopo,boxGrids.allGridsInGeoPoints,boxGrids.effectiveRadii)

In [ ]:
Nx,Nz=size(seismicModel.ρ)

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in boxGrids.allGridsInCartesian[:,1]]*1.e-3
zvals = [p.xz[2] for p in boxGrids.allGridsInCartesian[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModel.Vpv;
    colormap = :seismic,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "Vpv model")
)
Colorbar(fig[1,2], hm, label="Vpv")
fig

# Reading models from Takashi and Paul

Paul is ready to give us more models whenever we want but the data are confidential and I cannot share on the Github

In [ ]:

rhoFiles=myListDir(dir; pattern=r"test_rho\d");
compositionFiles=myListDir(dir; pattern=r"test_c\d");
temperatureFiles=myListDir(dir; pattern=r"test_t\d");
wtrFiles=myListDir(dir; pattern=r"test_wtr\d");